### CPI components 


https://www.bls.gov/cpi/tables/relative-importance/home.htm

In [1]:
import sys
sys.path.append('../src')

import requests
import re

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
# Series stored as a dictionary
series = {'CUUR0000SA0': 'All items',
          'CUUR0000SAM': 'Medical care',
          'CUUR0000SAH': 'Housing',
          'CUUR0000SEFV': 'Food away from home',
          'CUUR0000SAF11': 'Food at home',
          'CUUR0000SAR': 'Recreation',
          'CUUR0000SAE1': 'Education',
          'CUUR0000SAT': 'Transportation',
          'CUUR0000SAA': 'Apparel',
          'CUUR0000SA0E': 'Energy',
          'CUUR0000SAE2': 'Communication'}

In [3]:
url = 'https://www.bls.gov/cpi/tables/relative-importance/2018.txt'
rel_imp = pd.read_csv(url, header=None, delim_whitespace=True)
r = requests.get(url)
data = r.text

wgts = {value: re.findall(value+r'\..*\s+(\d+\.\d+)\s+\d+\.\d+\s+\r', data)[0] 
        for key, value in series.items()}

In [4]:
# Start year and end year
years = (2017, 2019)
df = bls_api(series, years, bls_key)

Post Request Status: REQUEST_SUCCEEDED


In [5]:
wgt_date = pd.to_datetime('2018-12-01')
dates = {'Latest': df.index[-1], 'Previous': df.index[-13]}

In [6]:
d = {}

for name, date in dates.items():

    updated_wgts = {srs: ((df.loc[date, srs] / df.loc[wgt_date, srs])
                     * float(wgts[srs]) / 
                     (df.loc[date, 'All items'] / 
                      df.loc[wgt_date, 'All items']))
                    for key, srs in series.items()}
    
    d[name] = {srs: (df[srs].pct_change(12).loc[date] 
                        * (updated_wgts[srs]))
                  for key, srs in series.items()}

In [7]:
results = pd.DataFrame(d).drop('All items').round(2).sort_values('Latest', ascending=False)
results.to_csv(data_dir / 'cpi_comp.csv', index_label='name')

mo1  = df.index[-1].strftime('%b %Y')
write_txt(text_dir / 'cpi_mo1.txt', mo1)

mo2  = df.index[-13].strftime('%b %Y')
write_txt(text_dir / 'cpi_mo2.txt', mo2)

In [12]:
results

,Latest,Previous
Housing,1.22,1.18
Medical care,0.38,0.15
Food away from home,0.20,0.15
Recreation,0.10,0.01
Food at home,0.07,0.01
Education,0.07,0.08
Communication,-0.03,-0.01
Apparel,-0.07,-0.01
Transportation,-0.23,0.92
Energy,-0.31,0.71


In [36]:
ltdate = df.index[-1].strftime('%B %Y')
prevdate = df.index[-13].strftime('%B %Y')

In [45]:
ltd = {i: (results.iloc[i].name, results['Latest'].iloc[i]) for i in [0, 1, -2, -1]}

textd = {}
for idx, (name, val) in ltd.items():
    if val >= 0.05:
        textd[idx] = f'{name} contributed {val:.2f} percentage points to'
    elif val <= -0.04:
        textd[idx] = f'{name} subtracted {abs(val):.2f} percentage points from'
    else:
        textd[idx] = 'did not contribute significantly to'

In [46]:
prevd = {i: (results.iloc[i].name, results['Previous'].iloc[i]) for i in [0, 1, -2, -1]}

textd2 = {}
for idx, (name, val) in prevd.items():
    if val >= 0.05:
        textd2[idx] = f'a contribution of {val:.2f} percentage points'
    elif val <= -0.04:
        textd2[idx] = f'a reduction of {abs(val):.2f} percentage points'
    else:
        textd2[idx] = 'virtually no effect on inflation '

In [47]:
textd

{0: 'Housing contributed 1.22 percentage points to',
 1: 'Medical care contributed 0.38 percentage points to',
 -2: 'Transportation subtracted 0.23 percentage points from',
 -1: 'Energy subtracted 0.31 percentage points from'}

In [48]:
text = (f'In {ltdate}, {textd[0]} overall CPI inflation, compared to '+
        f'{textd2[0]} in {prevdate}. {textd[1]} overall inflation in '+
        f'{ltdate}, compared to {textd2[1]} in {prevdate}.')

write_txt(text_dir / 'cpicomp.txt', text)

In [49]:
text2 = (f'{textd[-1]} overall CPI inflation in {ltdate}, compared to '+
         f'{textd2[-1]} in {prevdate}. {textd[-2]} overall inflation, '+
         f'compared to {textd2[-2]} the previous year.')

write_txt(text_dir / 'cpicomp2.txt', text2)

In [50]:
text2

'Energy subtracted 0.31 percentage points from overall CPI inflation in October 2019, compared to a contribution of 0.71 percentage points in October 2018. Transportation subtracted 0.23 percentage points from overall inflation, compared to a contribution of 0.92 percentage points the previous year.'